### Importação das bibliotecas necessárias para a raspagem de dados.

In [ ]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSING = '/home/thiago/tcc_ufrj/PRE_PROCESSING'

## Acessando o site Open Street Map e capturando as rotas pendentes e as rotas finalizadas.

In [ ]:
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
time.sleep(3)
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Acessando a telas para efetuar os downloads - Renomeando os arquivo adicionando o nome dos usuários - Movendo para a pasta PRE_PROCESSING

In [ ]:
navegador = webdriver.Chrome(service=servico)
users = []
for list_route in list_routes:
    time.sleep(3)
    url = list_route[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    user_page_content = navegador.page_source
    site_user = BeautifulSoup(user_page_content, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in site_user):
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)        
        print(f'Rota PENDENTE: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)

    else:
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)
        print(f'Rota FINALIZADA: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)
time.sleep(3)
navegador.close()

In [ ]:
## Corrigindo o nome dos arquivos conforme seus usuários
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

files_to_rename_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
files_to_rename_gpx

for user, file_to_rename_gpx in zip(users, files_to_rename_gpx):
    if os.listdir(DOWNLOADS):
        old_path = os.path.join(DOWNLOADS, file_to_rename_gpx)        
        new_name = f"{file_to_rename_gpx.replace('.gpx', '')}__{user}.gpx"        
        new_path = os.path.join(DOWNLOADS,new_name)        
        #print(new_path)
        os.rename(old_path, new_path)

In [ ]:
# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
files_to_move_pre_processing = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for file_to_move_pre_processing in files_to_move_pre_processing:
    src_path = os.path.join(DOWNLOADS, file_to_move_pre_processing)
    dest_path = os.path.join(PRE_PROCESSING, file_to_move_pre_processing)
    try:
        os.rename(src_path,dest_path)
        print(f"Arquivo '{file_to_move_pre_processing}' movido para a pasta {PRE_PROCESSING}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{file_to_move_pre_processing}': {e}.")

## Executando de uma só vez - Entendo que essa execução deve se dar de uma única vez, pois pode haver alguma falha no momento de renomear os arquivos

In [22]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())
import re
import os

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.
DOWNLOADS = '/home/thiago/Downloads/'
PRE_PROCESSING = '/home/thiago/tcc_ufrj/PRE_PROCESSING'

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
time.sleep(3)
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
#navegador = webdriver.Chrome(service=servico)


## Baixando os arquivos 
navegador = webdriver.Chrome(service=servico)
users = []
for list_route in list_routes:
    time.sleep(3)
    url = list_route[0]
    navegador.get(url) #--> Exemplo onde usamos o Selenium somente com o [.get]

    user_page_content = navegador.page_source
    site_user = BeautifulSoup(user_page_content, 'html.parser')

    if any(td.find('span', attrs={'class': 'text-danger'}) for td in site_user):
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[4]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)        
        print(f'Rota PENDENTE: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)

    else:
        tb_user_name = navegador.find_element('xpath', '//*[@id="content"]/div[2]/div/table/tbody/tr[6]/td')
        user_name = tb_user_name.text
        user_name = re.sub(r'\s|\.|\(|\)','_',user_name)
        print(f'Rota FINALIZADA: {list_route} Usuário: {user_name}')
        navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
        users.append(user_name)
time.sleep(3)
navegador.close()


## Corrigindo o nome dos arquivos conforme seus usuários
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

files_to_rename_gpx = sorted([arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")], reverse=True)
files_to_rename_gpx

for user, file_to_rename_gpx in zip(users, files_to_rename_gpx):    
    old_path = os.path.join(DOWNLOADS, file_to_rename_gpx)        
    new_name = f"{file_to_rename_gpx.replace('.gpx', '')}__{user}.gpx"        
    new_path = os.path.join(DOWNLOADS,new_name)            
    os.rename(old_path, new_path)


# Movendo os arquivos para a pasta PRE_PROCESSING
time.sleep(3)
files_to_move_pre_processing = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".gpx")]
for file_to_move_pre_processing in files_to_move_pre_processing:
    src_path = os.path.join(DOWNLOADS, file_to_move_pre_processing)
    dest_path = os.path.join(PRE_PROCESSING, file_to_move_pre_processing)
    try:
        os.rename(src_path,dest_path)
        print(f"Arquivo '{file_to_move_pre_processing}' movido para a pasta {PRE_PROCESSING}.")
    except Exception as e:
        print(f"Erro ao mover o arquivo: '{file_to_move_pre_processing}': {e}.")

PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781730
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781729
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781728
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781727
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781726
PENDING: https://www.openstreetmap.org/user/dragonpilot/traces/9781725
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781724
PENDING: https://www.openstreetmap.org/user/dragonpilot/traces/9781723
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781722
PENDING: https://www.openstreetmap.org/user/dragonpilot/traces/9781721
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781720
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781719
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781718
PENDING: https://www.openstreetmap.org/user/sunnypilot/traces/9781717
PENDING: https://

# VERIFICAR A FORMA DE LER OS ARQUIVOS QUE ESTÃO NA PASTA DE DOWNLOAD - CONVERTER PARA CSV OU PARQUET E APÓS ESSA CONVERSÃO LEVAR PARA O MINIO

## Lendo arquivos do Bucket - Transformando os arquivos .gpx em um dataframe

In [25]:
# pip install gpxpy
# pip install minio
import gpxpy 
import gpxpy.gpx 
import pandas as pd
import os
from minio import Minio
from minio.error import S3Error
BRONZE_LAYER = 'bronze'

In [26]:
minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

In [27]:
files_to_pre_processing = [arquivo for arquivo in os.listdir(PRE_PROCESSING) if arquivo.endswith(".gpx")] #--> Listando todos os arquivos da pasta preprocessamento com extensão .gpx
files_to_pre_processing

['9781876__Routes_from_sunnypilot_0_9_4_1-release__TOYOTA_COROLLA_TSS2_2019__.gpx',
 '9781724__Routes_from_sunnypilot_0_9_4_1-release__TOYOTA_PRIUS_TSS2_2021__.gpx',
 '9781659__Routes_from_top_0_9_4_Release_-_2023_07_28__TOYOTA_PRIUS_2017__.gpx',
 '9781416__Routes_from_sunnypilot_0_9_4_1-release__CHEVROLET_BOLT_EUV_2022__.gpx',
 '9781882__Routes_from_sunnypilot_0_9_3_1-release__HONDA_ACCORD_2018__.gpx',
 '9781720__Routes_from_sunnypilot_0_9_4_1-release__JEEP_GRAND_CHEROKEE_V6_2018__.gpx',
 '9781645__Routes_from_sunnypilot_0_9_4_1-release__HYUNDAI_SANTA_FE_2022__.gpx',
 '9781891__Routes_from_sunnypilot_0_9_4_1-release__HYUNDAI_IONIQ_5_2022__.gpx',
 '9781658__Routes_from_dragonpilot_2023_08_22__VOLKSWAGEN_SHARAN_2ND_GEN__.gpx',
 '9781497__Routes_from_sunnypilot_2023_08_23-dev__HONDA_CIVIC_2016__.gpx',
 '9781492__Routes_from_dragonpilot_2023_02_08__TOYOTA_RAV4_HYBRID_2019__.gpx',
 '9781498__Routes_from_sunnypilot_0_9_4_1-release__CHRYSLER_PACIFICA_2018__.gpx',
 '9781425__Routes_from_drago

In [29]:
## Criando arquivos CSV a partir dos arquivos .gpx, após a criação do CSV o .gpx correspondente é excluído
files_to_pre_processing = [arquivo for arquivo in os.listdir(PRE_PROCESSING) if arquivo.endswith(".gpx")] #--> Listando todos os arquivos da pasta pré-processamento com extensão .gpx

for file_to_pre_processing in files_to_pre_processing:
    file_path = os.path.join(PRE_PROCESSING, file_to_pre_processing) # --> Crie o caminho completo para o arquivo GPX

    try:
        with open(file_path, 'r', encoding='utf-8') as gpx_file:
            gpx = gpxpy.parse(gpx_file)

        info_rota = [] #--> Lista que servirá de apoio para converter o arquivo .gpx em uma lista
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    info_rota.append({
                        'latitude': point.latitude,
                        'longitude': point.longitude,
                        'elevacao' : point.elevation,
                        'time_point' : point.time
                    })
        file_to_pre_processing_csv = file_to_pre_processing.replace('.gpx','')
        info_rota_df = pd.DataFrame(info_rota)
        info_rota_df.to_csv(f'{PRE_PROCESSING}/{file_to_pre_processing_csv}.csv', index=False)        
        os.remove(file_path)    
    except Exception as e:
        print(f"Erro ao converter o '{file_to_pre_processing}': {e}.") #--> Exibindo o erro

Erro ao converter o '9781876__Routes_from_sunnypilot_0_9_4_1-release__TOYOTA_COROLLA_TSS2_2019__.gpx': Error parsing XML: no element found: line 3362, column 17.


In [ ]:
#files_to_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".csv")]
files_to_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".parquet")]
len(files_to_datalake)

In [ ]:
## Aqui os arquivos com a extensão .csv saem da pasta PRE_PROCESSING para a primeira camada (bronze) no MinIO
#files_to_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".csv")]
files_to_datalake = [files for files in os.listdir(PRE_PROCESSING) if files.endswith(".parquet")]
for name_file in files_to_datalake:
    local_path = os.path.join(PRE_PROCESSING, name_file)    
    if os.path.isfile(local_path):
        try:
            minioclient.fput_object(BRONZE_LAYER, name_file, local_path)
            print(f"Arquivo {name_file} enviado com sucesso para o bucket.")
            os.remove(local_path) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta download
        except S3Error as e:
            print(f"Erro ao enviar o arquivo: {name_file} -> Erro: {e}")     

In [ ]:
with open('PRE_PROCESSING/9632214__Routes_from_sunnypilot_2023_08_19-dev__SUBARU_IMPREZA_LIMITED_2019__.gpx', 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [ ]:
gpx_file = '9632229__Routes_from_sunnypilot_2023_08_19-dev__HYUNDAI_SANTA_FE_2019__.gpx'

In [ ]:
gpx_file = str(gpx_file)

In [ ]:
type(gpx_file)

In [ ]:
with open(gpx_file, 'r', encoding='utf-8') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [ ]:
gpx

In [ ]:
type(gpx)

In [ ]:
# Para retornar a quantidade de pontos (lat+lon+ele) em um arquivo arquivo '.gpx' podemos usar get
# Documentação: https://pypi.org/project/gpxpy/
gpx.get_track_points_no()

In [ ]:
# Para retornar a faixa de altitude, a fim de se obter extremos de maior elevação e menor elevação no trajeto percorrido - os valores apresentados são em metros acima do nível do mar
gpx.get_elevation_extremes()

In [ ]:
# Não endendi para que isso serve
gpx.get_uphill_downhill()

In [ ]:
# Obtendo o nome do criador da rota - Entretanto no nosso caso quando usamos esse comando temos uma informação generica - Foi necessário obter o nome do criador da rota com o WebScraping
creator = gpx.creator
creator

In [ ]:
# Para exibir o conteudo do arquivo .gpx em formato xml
print(gpx.to_xml()[:1000])

In [ ]:
# É possivel verificar quantas rotas/trilhas nosso arquivo .gpx possui
len(gpx.tracks)

In [ ]:
# Como no nosso exemplo acima só temos 1 rota/trilha efetuada, podemos acessar por meio de indice no python, 
# Nesse caso se passa o valor [0] para ter uma precisão exata da rota em que estamos trabalhando
gpx.tracks[0]

In [ ]:
# Agora vamos acessar os segmentos da nossa rota/trilha
gpx.tracks[0].segments[0]

In [ ]:
# Agora podemos acessar os pontos de dados individuais dentro do nosso gpx acessando a matriz de pontos. Aqui tambem podemo0s ver o nome das propriedades do arquivo como elevation e time
gpx.tracks[0].segments[0].points[:10]

In [ ]:
# Aqui acessamos as tres camadas: Track - Segments e Point e o passamos para um array em forma de dicionário. Note que podemos acessar cada atributo. No meu caso estou pegando latitude, Longitude, Elevação e a hora de cada ponto (entenda-se como ponto Lat+Long+Ele)
info_rota = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            info_rota.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevacao' : point.elevation,
                'time_point' : point.time
            })

In [ ]:
# Exibindo somente os 3 primeiros resultados
info_rota[:3]

In [ ]:
type(info_rota)

In [ ]:
# Aqui podemos transformar essa informação em um dataframe com a biblioteca pandas
info_rota_df = pd.DataFrame(info_rota)
info_rota_df.head()

In [ ]:
type(info_rota_df)

In [ ]:
# Aqui salvamos o pandas dataframe localmente
info_rota_df.to_csv('pandas_info_rota.csv', index=False)

# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

In [ ]:
# Aqui salvamos o pandas dataframe localmente
info_rota_df.to_parquet('pd_parquet_info_rota.parquet', index=False)

# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

## Lendo o bucket e transformando os arquivos encontrados lá em DF Pandas

In [38]:
# pip install minio
from minio import Minio
from minio.error import S3Error
from io import StringIO
import pandas as pd

BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

In [39]:
info_rota_csv = []
arquivos_rotas_gpx_csv = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(BRONZE_LAYER) if arquivo_gpx.object_name.endswith(".csv")]

In [79]:
blob_rota_csv = minioclient.get_object(BRONZE_LAYER, '9781413__Routes_from_dragonpilot_2023_07_05__TOYOTA_COROLLA_TSS2_2019__.csv')
rota_csv = blob_rota_csv.data
rota_csv = rota_csv.decode("ISO-8859-1")
pre_df = StringIO(rota_csv)
df = pd.read_csv(pre_df)

In [80]:
df.head()

,latitude,longitude,elevacao,time_point
0,24.897371,121.139635,214.256,2023-08-29 00:34:24.699000+00:00
1,24.897365,121.139613,214.281,2023-08-29 00:34:24.799000+00:00
2,24.897360,121.139591,214.299,2023-08-29 00:34:24.899000+00:00
3,24.897354,121.139569,214.328,2023-08-29 00:34:24.999000+00:00
4,24.897348,121.139548,214.329,2023-08-29 00:34:25.099000+00:00


In [82]:
df['time_point'] = pd.to_datetime(df['time_point'])
df['data'] = df['time_point'].dt.date
df['hora'] = df['time_point'].dt.strftime('%H:%M:%S')
df = df.drop(columns=['time_point'])


In [83]:
df.head()

,latitude,longitude,elevacao,data,hora
0,24.897371,121.139635,214.256,2023-08-29,00:34:24
1,24.897365,121.139613,214.281,2023-08-29,00:34:24
2,24.897360,121.139591,214.299,2023-08-29,00:34:24
3,24.897354,121.139569,214.328,2023-08-29,00:34:24
4,24.897348,121.139548,214.329,2023-08-29,00:34:25


In [84]:
import re
import numpy as np


text = "9781892__Routes_from_sunnypilot_0_9_3_1-release__HONDA_ACCORD_2018__.gpx"

# Definindo o padrão de regex
pattern = r'(.*?)__'

# Encontrando todas as correspondências no texto
matches = re.findall(pattern, text)

# Extraindo o nome do carro
username = matches[-1]
id = matches[0]

print(f'Id: {id} - username: {username}')


Id: 9781892 - username: HONDA_ACCORD_2018


In [85]:
num_df = len(df)
num_df

1500

In [86]:
repeated_id = np.tile(id, num_df)
repeated_user = np.tile(username, num_df)


In [87]:
df['id'] = repeated_id
df['username'] = repeated_user


In [88]:
# Reordenando as colunas
desired_order = ['id', 'username', 'latitude', 'longitude', 'elevacao', 'data', 'hora']
df = df.reindex(columns=desired_order)

In [89]:
df.head()

,id,username,latitude,longitude,elevacao,data,hora
0,9781892,HONDA_ACCORD_2018,24.897371,121.139635,214.256,2023-08-29,00:34:24
1,9781892,HONDA_ACCORD_2018,24.897365,121.139613,214.281,2023-08-29,00:34:24
2,9781892,HONDA_ACCORD_2018,24.897360,121.139591,214.299,2023-08-29,00:34:24
3,9781892,HONDA_ACCORD_2018,24.897354,121.139569,214.328,2023-08-29,00:34:24
4,9781892,HONDA_ACCORD_2018,24.897348,121.139548,214.329,2023-08-29,00:34:25


In [6]:
df.to_csv('arquivo.csv', index=False, sep=';')

## Códigos não mais usados

In [ ]:
## Aqui os arquivos saem da pasta download para a pasta DATALAKE_STAGE

# files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
# for name_files in files_to_move:
#     origin_path = os.path.join(DOWNLOADS, name_files)
#     destiny_path = os.path.join(DATALAKE_STAGE, name_files)
#     os.rename(origin_path, destiny_path)
#     print(f"Arquivo {name_files} movido para {destiny_path}")

In [ ]:
## Aqui os arquivos saem da pasta DATALAKE_STAGE para o bucket MinIO

# for item in os.listdir(DATALAKE_STAGE):
#     local_path = os.path.join(DATALAKE_STAGE, item)
#     print(local_path)
# 
#     if os.path.isfile(local_path):
#         try:
#             minioclient.fput_object(DATA_LAKE, item, local_path)
#             print(f"Arquivo {item} enviado com sucesso para o bucket.")
#         except S3Error as err:
#             print(f"Erro ao enviar o arquivo {item}: {err}")

In [ ]:
# ## DADOS PARA INSERIR NA TABELA DE CONTROLE USANDO O BEAUTIFULSOAP ## #
# navegador = requests.get('https://www.openstreetmap.org/user/sunnypilot/traces/9255974')
# site = BeautifulSoup(navegador.text, 'html.parser')
# data_trace = site.find('div', attrs={'class': 'content-body'})
# table_trace = data_trace.find('table')
# 
# filename_row = table_trace.find('th', string='Filename:').parent
# owner_row = table_trace.find('th', string='Owner:').parent
# uploaded_row = table_trace.find('th', string='Uploaded:').parent
# 
# filename = filename_row.find('td').text.strip().replace('(download)','').strip()
# owner = owner_row.find('td').text.strip()
# uploaded = uploaded_row.find('td').text.strip()

In [ ]:
# ## ESTE TRECHO TEM A FINALIDADE DE BAIXAR OS ARQUIVOS ## #
#list_finished_routes
#list_pending_routes

# //*[@id="content"]/div[2]/div/table/tbody/tr[6]/td - finalizado
# //*[@id="content"]/div[2]/div/span = PENDENTE
# //*[@id="content"]/div[2]/div/table/tbody/tr[4]/td - pendente

# ## Trecho descontinuado -- Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
# navegador = webdriver.Chrome(service=servico)
# for list_route in list_routes:
#     sleep(3)
#     url = list_route[0]
#     navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
#     navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
# navegador.close()

In [ ]:
# # pip install pyspark
# from pyspark.sql import SparkSession


# # Criando as variáveis que serão utilizadas no Spark
# appname = 'tcc-project'
# master = 'local'

# # Criando a sessão Spark
# spark = SparkSession.builder\
#     .appName(appname)\
#     .master(master)\
#     .getOrCreate()
# 
# spark


# # Coalesce o DataFrame para um único arquivo
# df_single_file = df.coalesce(1)
# # Salve o DataFrame coalesced como um único arquivo CSV
# df_single_file.write.csv(csv_file_path, header=True, mode="overwrite")

# Dessa forma o spark gera um csv particionado
# df.write.csv('pyspark_dataframe.csv', header=True, mode="overwrite")